In [1]:
import pandas as pd
from thefuzz import fuzz    # pip install thefuzz
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm


# add web scraped charts to dataset

In [2]:
#data = pd.read_csv('data_cut.csv')
charts = pd.read_csv('charts.csv')

In [32]:
data = pd.read_csv('data_cut_missing_lyrics.csv')

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,time_signature,year,country_code,country,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics
0,USSM18600196,"['hard-rock', 'metal', 'rock', 'swedish']",the final countdown,europe,The Final Countdown (Expanded Edition),1986,year,spotify:track:3MrRksHupTVEQ7YbA0FsZK,3MrRksHupTVEQ7YbA0FsZK,1483.0,...,4,1986,US,United States,1041149,5591606,"['80s', 'rock', 'hard rock', 'classic rock', '...",the final countdown,https://genius.com/Europe-the-final-countdown-...,"We're leaving together, but still, it's farewe..."
1,USMC18925673,"['hard-rock', 'rock', 'singer-songwriter', 'so...",free fallin',tom petty,Full Moon Fever,1989-01-01,day,spotify:track:5tVA6TkbaAH9QMITTQRrNv,5tVA6TkbaAH9QMITTQRrNv,NaN,...,4,1989,US,United States,712744,3746238,"['classic rock', 'rock', '80s', 'Tom Petty', '...",free fallin',https://genius.com/Tom-petty-free-fallin-lyrics,"She's a good girl, loves her mama\nLoves Jesus..."
2,USPR39402222,"['metal', 'rock']",wanted dead or alive,bon jovi,Slippery When Wet,1986-08-16,day,spotify:track:0oerlffJSzhRVvtDfLcp3N,0oerlffJSzhRVvtDfLcp3N,NaN,...,4,1986,US,United States,697080,3925244,"['rock', '80s', 'classic rock', 'hard rock', '...",wanted dead or alive,https://genius.com/Bon-jovi-wanted-dead-or-ali...,"It's all the same, only the names will change\..."
3,USSM10100176,"['pop', 'power-pop', 'rock']",your love,the outfield,Play Deep,1985-06-29,day,spotify:track:0WoFs3EdGOx58yX5BtXvOa,0WoFs3EdGOx58yX5BtXvOa,NaN,...,4,1985,US,United States,650855,4412590,"['80s', 'rock', 'classic rock', 'pop', 'new wa...",your love,https://genius.com/The-outfield-your-love-lyrics,Josie's on a vacation far away\nCome around an...
4,USRH10901907,"['dance', 'pop', 'rock', 'synth-pop']",she drives me crazy,fine young cannibals,The Raw & The Cooked,1988-01-01,day,spotify:track:4d6eqRtpDX7tydHJGDZUBQ,4d6eqRtpDX7tydHJGDZUBQ,1841.0,...,4,1988,US,United States,473306,2283802,"['80s', 'pop', 'new wave', 'rock', 'british']",she drives me crazy,https://genius.com/Fine-young-cannibals-she-dr...,I can't stop the way I feel\nThings you do don...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54070,USA371410493,"['alt-rock', 'alternative', 'hard-rock', 'hard...",power failure,"minutemen,black flag",MINUTEFLAG,1986,year,spotify:track:3LZKwm06rS3ghseQp7k2gD,3LZKwm06rS3ghseQp7k2gD,NaN,...,4,1986,US,United States,1010,1558,[],power failure,no url,no lyrics
54071,USA371666845,['german'],run to vegas,tangerine dream,The Best of Tangerine Dream,1989,year,spotify:track:6bL6OxpPyj05A6BEAvFmcr,6bL6OxpPyj05A6BEAvFmcr,NaN,...,3,1989,US,United States,2125,4927,"['electronic', 'instrumental', 'ambient', 'cin...",run to vegas,no url,no lyrics
54072,USA3M0600028,['indie'],open wide,my dad is dead,"Peace, Love and Murder",1987-02-24,day,spotify:track:4mcaTgy5Zxgr0w47o5DZMR,4mcaTgy5Zxgr0w47o5DZMR,NaN,...,4,1987,US,United States,3136,10530,"['indie', 'alternative rock', 'indie rock', 'U...",open wide,no url,no lyrics
54073,USRO28820683,"['blues', 'soul']",steal away,walter wolfman washington,Out Of The Dark,1988-01-01,day,spotify:track:5efaq8Gu2FuMTeFpNEIyzl,5efaq8Gu2FuMTeFpNEIyzl,NaN,...,4,1988,US,United States,1244,2258,[],steal away,no url,no lyrics


In [37]:
charts.head()

,Artist,Title,Label,Peak,Weeks
0,16 Bit,Changing Minds,Ariola,43,1
1,16 Bit,Where Are You?,Ariola,50,1
2,2 Plus 1,"Easy Come, Easy Go",Autobahn,40,3
3,49ers,Touch Me,BCM,47,1
4,5 Star,Can't Wait Another Minute,RCA,55,1


In [38]:
#function to match song and artist
def match_song(song1, artist1, song2, artist2):
    artists = artist1.lower().split(',')
    for artist in artists:
        if fuzz.ratio(artist, artist2.lower()) > 70:
            if fuzz.ratio(song1.lower(), song2.lower()) > 50:
                return True
            else:
                return False
        else:
            return False

In [41]:
#match and only return index, peak and week
def process_data(i, song1, artist1, charts):
    peak = 0
    week = 0
    for j in range(len(charts)):
        if match_song(song1, artist1, charts.iloc[j]['Title'], charts.iloc[j]['Artist']):
            peak = (charts.iloc[j]['Peak'])
            week = (charts.iloc[j]['Weeks'])
            break
    return i, peak, week
    
    

In [42]:
#array of song names
names = data['name'].values
artists = data['artists'].values


In [43]:
#for displaying progress during runtime
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [44]:
#match with parallel processing
with tqdm_joblib(tqdm(desc="My calculation", total=len(data))) as progress_bar:
    results = Parallel(n_jobs=-1)(delayed(process_data)(i, names[i], artists[i], charts) for i in range(len(data)))

My calculation: 100%|██████████| 54075/54075 [29:26<00:00, 30.61it/s]


In [45]:
results

[(0, 24, 1),
 (1, 0, 0),
 (2, 58, 1),
 (3, 55, 1),
 (4, 59, 1),
 (5, 95, 1),
 (6, 81, 1),
 (7, 27, 1),
 (8, 0, 0),
 (9, 57, 1),
 (10, 0, 0),
 (11, 62, 1),
 (12, 46, 1),
 (13, 0, 0),
 (14, 22, 1),
 (15, 0, 0),
 (16, 0, 0),
 (17, 47, 1),
 (18, 0, 0),
 (19, 0, 0),
 (20, 22, 1),
 (21, 0, 0),
 (22, 0, 0),
 (23, 32, 1),
 (24, 0, 0),
 (25, 0, 0),
 (26, 74, 1),
 (27, 0, 0),
 (28, 0, 0),
 (29, 0, 0),
 (30, 0, 0),
 (31, 40, 1),
 (32, 52, 1),
 (33, 0, 0),
 (34, 0, 0),
 (35, 34, 1),
 (36, 0, 0),
 (37, 0, 0),
 (38, 0, 0),
 (39, 0, 0),
 (40, 0, 0),
 (41, 0, 0),
 (42, 46, 1),
 (43, 52, 1),
 (44, 31, 1),
 (45, 0, 0),
 (46, 0, 0),
 (47, 0, 0),
 (48, 0, 0),
 (49, 0, 0),
 (50, 0, 0),
 (51, 0, 0),
 (52, 54, 1),
 (53, 60, 1),
 (54, 37, 1),
 (55, 0, 0),
 (56, 0, 0),
 (57, 0, 0),
 (58, 0, 0),
 (59, 0, 0),
 (60, 0, 0),
 (61, 0, 0),
 (62, 0, 0),
 (63, 0, 0),
 (64, 0, 0),
 (65, 0, 0),
 (66, 0, 0),
 (67, 0, 0),
 (68, 0, 0),
 (69, 0, 0),
 (70, 0, 0),
 (71, 53, 1),
 (72, 0, 0),
 (73, 0, 0),
 (74, 28, 1),
 (75, 0, 

In [46]:
#add peak and week to data
for i in range(len(data)):
    data.loc[i, 'peak'] = results[i][1]
    data.loc[i, 'week'] = results[i][2]

In [49]:
#delete columns chart_peeks and chart_weeks
data = data.drop(['chart_peak', 'chart_weeks'], axis=1)

In [50]:
#rename columns peak and week
data = data.rename(columns={'peak': 'chart_peak', 'week': 'chart_weeks'})

In [51]:
#make both columns integer
data['chart_peak'] = data['chart_peak'].astype(int)
data['chart_weeks'] = data['chart_weeks'].astype(int)

In [52]:
#store data to csv
# data.to_csv('data_cut_lyrics_charts.csv', index=False)

In [4]:
#count songs with chart peak > 0
data[data['chart_peak'] > 0].shape[0]

2686

In [7]:
charts.shape

(3268, 5)